# Numérisation

## Introduction

Il existe différentes filières pour obtenir un régulateur numérique:

1. En numérisant un régulateur analogique obtenu par:
    - Méthode expérimentale (Ziegler & Nichols)
    - Synthèse analogique partant d'un essai harmonique analogique ou via un modèle physique
2. En effectuant une synthèse directe à partir:
    - Du modèle obtenu par échantillonnage du modèle physique
    - D'un essai harmonique discret
    - D'un modèle obtenu par identification paramétrique

Ce chapitre se concentre sur la numérisation d'un régulateur analogique.

## Numérisation d'un régulateur analogique

### Echantillonnage du régulateur analogique

Cette première méthode se base sur l'approximation qu'un bloc $A-D$ suivi immédiatement d'un bloc $D-A$ ne crée aucune distorsion entre le signal d'entrée et de sortie. Ceci est bien évidemment faux, comme montré au chapitre 6, car le convertisseur $D-A$ génère un retard d'une demi-période d'échantillonnage.

On obtient alors le schéma suivant:

<img src="Images/Chapter8/control_loop_stoz.png">

#### Exemple

Soit un PID mixte analogique dont la dérivée est filtrée:

$$ K(s) = K_p \left( 1 + \frac{1}{T_i s} + \frac{T_d s}{\frac{T_d}{N}s+1} \right) $$

En appliquant la transformation par échantillonnage, on obtient:

$$ K'(z) = K_p \left( 1 + \frac{\frac{h}{T_i}}{z-1} + \frac{N(z-1)}{z-e^{-N\frac{h}{T_d}}} \right) $$
***

### Approximations numériques

Il est possible, en repartant des équations différentielles, d'obtenir d'autres approximations.

#### Première méthode d'Euler

La dérivée d'une grandeur est donnée par, à un instant $k$:

$$ \dot{u}(kh) = \lim_{h \rightarrow 0} \frac{u(kh+h) - u(kh)}{h} $$

En utilisant la transformée en $z$, on obtient:

$$ \mathscr{Z} \left\{ \dot{u}(kh) \right\} = \frac{z U(z) - U(z)}{h} = \frac{z-1}{h} U(z) $$

Comme la transformée de Laplace de la dérivée d'une grandeur analogique correspond à:

$$ \mathscr{L} \left\{ \dot{u}(t) \right\} = s U(s) $$

On a la transformation suivante:

$$ s = \frac{z-1}{h} $$

#### Exemple

En appliquant la méthode au régulateur PID, on obtient:

$$ K'(z) = K_p \left( 1 + \frac{\frac{h}{T_i}}{z-1} + \frac{N(z-1)}{z-(1-N\frac{h}{T_d})} \right) $$
***

#### Seconde méthode d'Euler

La dérivée d'une grandeur est donnée par, à un instant $k$:

$$ \dot{u}(kh) = \lim_{h \rightarrow 0} \frac{u(kh) - u(kh-h)}{h} $$

En utilisant la transformée en $z$, on obtient:

$$ \mathscr{Z} \left\{ \dot{u}(kh) \right\} = \frac{U(z) - z^{-1} U(z)}{h} = \frac{1-z^{-1}}{h} U(z) $$

Comme la transformée de Laplace de la dérivée d'une grandeur analogique correspond à:

$$ \mathscr{L} \left\{ \dot{u}(t) \right\} = s U(s) $$

On a la transformation suivante:

$$ s = \frac{1-z^{-1}}{h} = \frac{z-1}{hz} $$

#### Exemple

En appliquant la méthode au régulateur PID, on obtient:

$$ K'(z) = K_p \left( 1 + \frac{\frac{h}{T_i}z}{z-1} + \frac{N(z-1)}{(1+N\frac{h}{T_d})z-1} \right) $$
***

#### Méthode de Tustin

Cette fois-ci, on part de l'équation différentielle dont on intègre $n$ fois chaque membre afin de faire disparaître toute dérivée (avec $n$ l'ordre du système).

La méthode de Tustin consiste à approximer les intégrales par la règle des trapèzes:

\begin{align}
  i(kh) &= \int_0^{kh}u(\tau)d\tau \\
  i(kh) &= i(kh-h) + \frac{h}{2} \left( u(kh-h) + u(kh) \right)
\end{align}

Par la transformée en $z$, on obtient:

\begin{align}
  I(z) - z^{-1} I(z) &= \frac{h}{2} \left( z^{-1} U(z) + U(z) \right) \\
  \frac{z-1}{z} I(z) &= \frac{h}{2} \left( \frac{1+z}{z} \right) U(z) \\
  I(z) &= \frac{h}{2} \left( \frac{z+1}{z-1} \right) U(z)
\end{align}

Comme la transformée de Laplace de l'intégrale d'une grandeur analogique correspond à:

$$ \mathscr{L} \left\{ \int u(t) \right\} = \frac{1}{s} U(s) $$

On a la transformation suivante:

$$ s = \frac{2}{h} \left( \frac{z-1}{z+1} \right) $$

#### Exemple

En appliquant la méthode au régulateur PID, on obtient:

$$ K'(z) = K_p \left( 1 + \frac{\frac{h}{2 T_i}(z+1)}{z-1} + \frac{N(z-1)}{(1+N\frac{h}{2 T_d})z-(1-N\frac{h}{2 T_d})} \right) $$
***

Le schéma ci-dessous permet de visualiser la transformation du demi-plan gauche $s$ dans le plan $z$.

<img src="Images/Chapter8/numerisation_stoz_plan.png">

Les transformations montrent:

- La première méthode d'Euler peut transformer des pôles stables en $s$ en des pôles instables en $z$; elle est donc à déconseiller
- La seconde méthode d'Euler transforme un régulateur BIBO stable en un autre régulateur BIBO stable; il permet en plus d'éviter d'avoir des modes sonnettes dus à des pôles dans la partie gauche du cercle unité
- La méthode de Tustin transforme l'entièreté du demi-plan gauche à l'intérieur du cercle unité complet; un régulateur BIBO stable le restera

#### Commentaires

Il faut noter les problématiques suivantes:

- pour espérer obtenir un comportement correct, il faut que la période d'échantillonnage soit très petite
- une trop petite période d'échantillonnage demande du matériel spécifique coûteux, et peu engendrer des problèmes numériques
- le retard généré par les convertisseurs ainsi que la quantification n'étant pas pris en compte, il faut s'attendre à une dégradation des performances
- les approximations numériques commettent des erreurs équivalentes à des erreurs de modélisations; les marges de stabilité sont dégradées

On peut essayer de réduire l'effet des convertisseurs en ajoutant un retard pur analogique équivalent à une demi-période d'échantillonnage dans le modèle analogique du système afin de modéliser l'ensemble "convertisseurs/système".

## Régulateur PID numérique

### Forme position

Le PID numérique "standard" est décrit par la fonction de transfert discrète suivante, en utilisant la seconde méthode d'Euler:

$$ K'(z) = K_p \left( 1 + \frac{\frac{h}{T_i}z}{z-1} + \frac{N(z-1)}{(1+N\frac{h}{T_d})z-1} \right) $$

En voici une représentation en schéma bloc:

<img src="Images/Chapter8/mixt_PID.png">

Pour rappel, les paramètres sont:

- $K_p$ : gain proportionnel, augmente la rapidité et la précision mais diminue la stabilité
- $T_i$ : constante de temps d'intégration, augmente la précision mais diminue la stabilité
- $T_d$ : constante de temps de dérivation, augmente la stabilité
- $N$ : coefficient de filtrage sur la dérivée, $\in [3 \dots 20]$

Les régulateurs P, PI et PD sont obtenus en supprimant la branche non voulue:

\begin{align}
  K(z) &= K_p \\
  K(z) &= K_p \left( 1 + \frac{\frac{h}{T_i}z}{z-1} \right) \\
  K(z) &= K_p \left( 1 + \frac{N(z-1)}{(1+N\frac{h}{T_d})z-1} \right)
\end{align}

### Forme incrémentale

La forme incrémentale est parfois implémentée, permettant de calculer $\Delta u(kh)$:

\begin{align}
  \Delta U(z) &= \mathscr{Z}\left\{ \Delta u(kh) \right\} = (1 - z^{-1}) U(z) = (1 - z^{-1}) K(z) E(z) \\
  &= \frac{z-1}{z} K_p \left( 1 + \frac{\frac{h}{T_i}z}{z-1} + \frac{N(z-1)}{(1+N\frac{h}{T_d})z-1} \right) E(z) \\
  &= K_p \left( \frac{z-1}{z} + \frac{h}{T_i} + \frac{N(z^2-2z+1)}{(1+N\frac{h}{T_d})z^2-z} \right) E(z)
\end{align}

On en tire la fonction de transfert du régulateur incrémental:

$$ K_i(z) = \frac{\Delta U(z)}{E(z)} = K_p \left( \frac{z-1}{z} + \frac{h}{T_i} + \frac{N(z^2-2z+1)}{(1+N\frac{h}{T_d})z^2-z} \right) $$

Il faut bien évidemment, avant d'appliquer la commande au système, calculer $u(kh) = u(kh-h) + \Delta u(kh)$.

### Forme à dérivation séparée

Un gros inconvénient de la dérivée appliquée à l'erreur est qu'elle génère de grosses commandes dues à un échelon de consigne. Une façon de filtrer ces sauts est d'appliquer la dérivée directement sur la sortie y(kh).

<img src="Images/Chapter8/mixt_PID_separated_derivative.png" width="800">

### Forme non filtrée

Le PID théorique est en général étudié sans filtre sur la dérivée. Sa fonction de transfert analogique est:
    
$$ K(s) = K_p \left( 1 + \frac{1}{T_i s} + T_d s \right) $$

Cependant, ce PID analogique est irréalisable en pratique car la dérivée est non causale. Par contre, sa forme numérique, par la seconde méthode d'Euler, donne:

$$ K(z) = K_p \left( 1 + \frac{\frac{h}{T_i}z}{z-1} + \frac{T_d}{h} \frac{z-1}{z} \right) $$

Cette fonction de transfert est réalisable en pratique.

### Structures

Les structures vues jusqu'à présent sont mixtes. Il existe différentes structures, dont la parallèle et la série.

#### Parallèle

La structure parallèle découple les actions. Sa fonction de transfert discrète est:

$$ K(z) = K_p + \frac{\frac{h}{T_i}z}{z-1} + \frac{T_d}{h} \frac{z-1}{z} $$

<img src="Images/Chapter8/parallel_PID.png" width="800">

#### Série

La structure série est donnée par:

$$ K(z) = K_p \left( 1 + \frac{\frac{h}{T_i}z}{z-1} \right) \left( 1 + \frac{T_d}{h} \frac{z-1}{z} \right) $$

<img src="Images/Chapter8/serial_PID.png">

Même si cette structure est peu rencontrée en pratique, elle possède un atout intéressant: les zéros de la fonction de transfert dépendent séparement des actions intégrales et dérivées! Ceci simplifiera grandement les calculs lors de la synthèse.

Cependant, comme on croise plus souvent la forme mixte, on doit pouvoir convertir la forme série en forme mixte.

La fonction de transfert peut s'écrire:

$$ K(z) = K_p \left( 1 + \frac{\frac{h}{T_i}z}{z-1} + \frac{T_d}{h} \frac{z-1}{z} + \frac{T_d}{T_i} \right) $$

Si $T_d$ est plus petit que $T_i$, la fonction de transfert devient celle d'une structure mixte. 

Par contre, si ce n'est pas le cas, on a:

\begin{align}
K(z) &= K_p \left( 1 + \frac{T_d}{T_i} + \frac{\frac{h}{T_i}z}{z-1} + \frac{T_d}{h} \frac{z-1}{z} \right) \\
     &= K_p \frac{T_i + T_d}{T_i} \left( 1 + \frac{\frac{h}{T_i + T_d}z}{z-1} + \frac{\frac{T_i T_d}{T_i + T_d}}{h} \frac{z-1}{z} \right) 
\end{align}

En redéfinissant les termes ainsi:

\begin{align}
  K'_p &= K_p \frac{T_i + T_d}{Ti} \\
  T'_i &= T_i + T_d \\
  T'_d &= \frac{T_i T_d}{T_i + T_d}
\end{align}

La fonction de transfert devient donc:

$$ K(z) = K'_p \left( 1 + \frac{\frac{h}{T'_i}z}{z-1} + \frac{T'_d}{h} \frac{z-1}{z} \right) $$

Elle correspond bien à une structure mixte. On peut donc profiter de sa propriété afin de choisir les actions intégrales et dérivées séparement, puis transformer les paramètres afin de pouvoir paramétrer les régulateurs industriels, souvent mixtes.

### Mesures anti-emballement de l'intégrateur

L'action intégrale possède la caractéristique d'accumuler l'erreur en fonction du temps. Elle a donc tendance, au fur et à mesure du temps, à augmenter la commande à fournir au système. 

Ceci peut causer des problèmes si le système ne réagit pas suffisamment vite pour une quelconque raison. En effet, l'intégrale augmentera l'action intégrale jusqu'à atteindre la saturation de l'actionneur. A partir de ce moment-là, même si l'intégrale continue d'augmenter, l'action réelle, elle, reste constante. On entre dans une zone non-linéaire.

Le problème qui arrive ensuite est le suivant: afin de retourner dans la zone linéaire de l'actuateur, il faut vider l'intégrale... ce qui ne peut se faire qu'en inversant le signe de l'erreur, et donc, en créant un dépassement de la consigne. Ce dépassement sera d'autant plus grand que l'intégrale était trop pleine. Le système en boucle fermé possèdera donc d'énormes oscillations.

Cet effet s'appelle l'emballement de l'intégrateur (en anglais, *integral windup* ou *reset windup*).

Il existe beaucoup de littérature sur le sujet, tellement ce problème est important. Nous nous restreindrons aux mesures les plus simples à comprendre et mettre en oeuvre. Ces méthodes sont appelées les mesures anti-emballement ou anti-saturation de l'intégrateur (en anglais, *antireset windup*).

On suppose que la saturation de l'actionneur est symétrique et décrite par:

$$
u_e(kh) = \left\{
\begin{array}{ll}
  \mu & \quad \text{si}\, u(kh) > \mu \\
  u(kh) & \quad \text{si}\, u(kh) \in [-\mu, \mu] \\
  -\mu & \quad \text{si}\, u(kh) < -\mu
\end{array}
\right.
$$

#### Première méthode
Cette première méthode est appelée intégration conditionnelle. Elle consiste à activer l'intégration lorsque l'écart est plus petit qu'une constante donnée:

$$
u_i(kh) = \left\{
\begin{array}{ll}
  0 & \quad \text{si}\, |e(kh)| > \epsilon \\
  u_i(kh-h) + \frac{h}{T_i}e(kh) & \quad \text{si}\, |e(kh)| \leq \epsilon
\end{array}
\right.
$$

Le souci de cette méthode est qu'il faut déterminer $\epsilon$ par simulation ou expérimentalement.

#### Seconde méthode
La seconde méthode consiste à geler l'intégration dès que la commande atteint la saturation $|u(kh)| > \mu$:

$$ u_i(kh) = u_i(kh-h) $$

$$ u(kh) = K_p \left( e(kh) + u_i(kh) + u_d(kh) \right) $$

Il existe d'autres méthodes plus complexes. On peut citer, par exemple:

- la mesure anti-emballement standard
- la méthode de Franklin-Powell
- la méthode d'Aström-Wittenmark